In [9]:
import math
import time
import numpy as np
import matplotlib.pyplot as plt
from random import uniform
from scipy.stats import chisquare

#plt.rcParams['figure.figsize'] = [8, 5]

TIME = 1
GRAVITY = np.array([0, 0, -0.01], dtype=float)
ZMIN = 0.002

In [6]:
class Particle:
    def __init__(self, a = [0, 0, 0]):
        self.mass = 1
        self.position = np.array(a, dtype=float) 
        self.velocity = np.array([0, 0, 0], dtype=float)
        self.force = np.array([0, 0, 0], dtype=float)
        #self.correction = 0.0
        
    def __str__(self):
        return "({:.3f}, {:.3f}, {:.3f})".format(self.position[0], self.position[1], self.position[2])
        
class Spring:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        self.length = np.linalg.norm(self.b.position - self.a.position)
        
    def __str__(self):
        return "{}, {}, {}".format(self.a, self.b, self.length)
    
class Poligon:
    def __init__(self, particles):
        self.particles = particles
        self.n = len(self.particles)
        self.springs = []
        for i in range(self.n - 1):
            for j in range(i + 1, self.n):
                self.springs.append(Spring(self.particles[i], self.particles[j]))
                
    def display(self): #only 2d
        x, y = [], []
        for particle in self.particles:
            x.append(particle.position[0])
            y.append(particle.position[1])
        plt.fill(x, y)
        #plt.axis('equal')
        plt.xlim(0, 800)
        plt.ylim(0, 400)
        plt.show()
        plt.clf()
        
    def update(self, time, gravity):
        #set gravity
        for particle in self.particles:
            particle.force = np.array(gravity)
        
        #use actual velocity
        for particle in self.particles:
            particle.force += particle.velocity
            
        #use springs
        for spring in self.springs:
            currentLength = np.linalg.norm(spring.b.position - spring.a.position)
            
            #dx = spring.b.position[0] - spring.a.position[0]
            #dy = spring.b.position[1] - spring.a.position[1]
            #dz = spring.b.position[2] - spring.a.position[2]
            #print(dx, dy, dz)
            #currentLength = np.sqrt(dx*dx + dy*dy + dz*dz)
            
            unit = spring.b.position - spring.a.position
            unit /= currentLength
            
            diff = spring.length - currentLength
            alpha = 0.1
            correction = diff * diff * alpha
            if diff < 0:
                correction *= -1
            #self.correction = correction
            unit = unit * correction
                
            spring.a.force -= unit
            spring.b.force += unit
            
        #use ground
        for particle in self.particles:
            if particle.position[2] < 0:
                particle.position[2] = 0
                particle.force[2] = 0
                #particle.force *= 0.99
                
        #update velocities
        for particle in self.particles:
            particle.velocity = particle.force * time
            
        #update positions
        for particle in self.particles:
            particle.position += particle.velocity * time
            
    def translate(self, vector):
        for particle in self.particles:
            particle.position += vector
         
    #rad!
    def rotate(self, xAlpha, yAlpha, zAlpha):
        cx = math.cos(xAlpha)
        sx = math.sin(xAlpha)
        cy = math.cos(yAlpha)
        sy = math.sin(yAlpha)
        cz = math.cos(zAlpha)
        sz = math.sin(zAlpha)

        Rx = np.array([[1,  0,  0, 0],
                       [0, cx,-sx, 0],
                       [0, sx, cx, 0],
                       [0,  0,  0, 1]])
        Ry = np.array([[ cy, 0, sy, 0],
                       [  0, 1,  0, 0],
                       [-sy, 0, cy, 0],
                       [  0, 0,  0, 1]])
        Rz = np.array([[cz,-sz, 0, 0],
                       [sz, cz, 0, 0],
                       [ 0,  0, 1, 0],
                       [ 0,  0, 0, 1]])

        R = Rz.dot(Ry).dot(Rx)
        
        for particle in self.particles:
            v = np.append(particle.position, 0)[np.newaxis]
            particle.position = R.dot(v.T).T[0][:3]
            
    def scale(self, a):
        t = np.array([[a, 0, 0],
                      [0, a, 0],
                      [0, 0, a]])
        for particle in self.particles:
            v = np.append(particle.position)[np.newaxis]
            particle.position = t.dot(v.T).T[0]

    def randrotate(self):
        #particles[0] is the pivot
        #we assume the origo is the center of the object
        #this means only apply this after translate(-c) !

        #random point on the sphere surface
        #hivatkozás!
        while True:
            r_point = np.array([np.random.normal() for _ in range(3)]) 
            if np.linalg.norm(r_point) != 0:
                break
        radius = np.linalg.norm(self.particles[0].position)
        r_point *= radius / np.linalg.norm(r_point)

        #rotate pivot onto the x axis (radius, 0, 0)
        if self.particles[0].position[1] != 0:
            temp_r = math.sqrt(self.particles[0].position[0]**2 + self.particles[0].position[1]**2)
            fi_z = np.arccos(self.particles[0].position[0] / temp_r)
            if self.particles[0].position[1] > 0:
                fi_z *= -1
            self.rotate(0, 0, fi_z)

        if self.particles[0].position[2] != 0:
            temp_r = math.sqrt(self.particles[0].position[0]**2 + self.particles[0].position[2]**2)
            fi_y = np.arccos(self.particles[0].position[0] / temp_r)
            if self.particles[0].position[2] > 0:
                fi_y *= -1
            self.rotate(0, -fi_y, 0)

        #random rotation arount the x axis
        fi_x = uniform(0, 2*math.pi)
        self.rotate(fi_x, 0, 0)

        #rotate pivot to the random point
        #1st rotate around the y axis until pivot.z = r_point.z
        if self.particles[0].position[2] != r_point[2]:
            a = np.array([self.particles[0].position[0], self.particles[0].position[2]])
            rx = math.sqrt(radius**2 - r_point[2]**2)
            b = np.array([rx, r_point[2]])
            a /= np. linalg.norm(a)
            b /= np. linalg.norm(b)
            fi_y = np.arccos(np.dot(a, b))
            if r_point[2] > 0:  
                fi_y *= -1
            poligon.rotate(0, fi_y, 0)

        #2nd rotate around the z axis
        if self.particles[0].position[1] != r_point[1]:
            a = np.array([self.particles[0].position[0], self.particles[0].position[1]])
            b = np.array([r_point[0], r_point[1]])
            a /= np. linalg.norm(a)
            b /= np. linalg.norm(b)
            fi_z = np.arccos(np.dot(a, b))
            if r_point[1] < 0:  
                fi_z *= -1
            poligon.rotate(0, 0, fi_z)

        #with np.printoptions(precision=3, suppress=True):
        #    print(r_point)
        #    print(self.particles[0].position) 

    def getTotalVelocityLength(self):
        totalVelocityLength = 0
        for particle in self.particles:
            totalVelocityLength += abs(np.linalg.norm(particle.velocity))
        return totalVelocityLength
            
    def isStopped(self):
        ground = 0
        for particle in self.particles:
            if particle.position[2] < ZMIN:
                ground += 1
        totalVelocityLength = 0
        for particle in self.particles:
            totalVelocityLength += abs(np.linalg.norm(particle.velocity))
        ground
        totalVelocityLength
        return ((ground > 2) and (totalVelocityLength < 0.13))

In [ ]:
#set data - tetrahedron

a = 50
height = 50

dropHeight = np.array([0, 0, height], dtype=float)

p = [[   0,                         0,                                         0],
     [ a/2,     a*math.sin(math.pi/3),                                         0],
     [-a/2,     a*math.sin(math.pi/3),                                         0],
     [   0, a/(2*math.sin(math.pi/3)), a*math.sin(math.pi/4)/math.sin(math.pi/3)]]

faces = [{0, 1, 2},
         {0, 1, 3},
         {0, 2, 3},
         {1, 2, 3}]

In [7]:
#set data - cube

a = 50
height = 50

dropHeight = np.array([0, 0, height], dtype=float)

p = [[0, 0, 0],
     [0, a, 0],
     [a, 0, 0],
     [a, a, 0],
     [0, 0, a],
     [0, a, a],
     [a, 0, a],
     [a, a, a]]

faces = [{0, 1, 2, 3},
         {0, 1, 4, 5},
         {0, 2, 4, 6},
         {1, 3, 5, 7},
         {2, 3, 6, 7},
         {4, 5, 6, 7}]

In [ ]:
#set data - octahedron

a = 50
height = 50

dropHeight = np.array([0, 0, height], dtype=float)

p = [[ 0,  0,   0],
     [ 0,  a,   a],
     [ 0, -a,   a],
     [ a,  0,   a],
     [-a,  0,   a],
     [ 0,  0, 2*a]]

faces = [{0, 1, 3},
         {0, 1, 4},
         {0, 2, 3},
         {0, 2, 4},
         {1, 3, 5},
         {1, 4, 5},
         {2, 3, 5},
         {2, 4, 5}]

In [ ]:
#droptest

particles = []
for a in p:
    particles.append(Particle(a))

poligon = Poligon(particles)
poligon.translate(dropHeight)

i = 0
while (not poligon.isStopped()) and i < 10000:
    poligon.update(TIME, GRAVITY)
    i += 1
print("{}, {}".format(i, Poligon.isStopped(poligon)))
#print(poligon.correction)

In [ ]:
#droptest with total velocity length graph

particles = []
for a in p:
    particles.append(Particle(a))

poligon = Poligon(particles)
poligon.translate(dropHeight)

i = 0
v = []
#while (not poligon.isStopped()) and i < 10000:
while i < 10000:
    poligon.update(TIME, GRAVITY)
    v.append(poligon.getTotalVelocityLength())
    i += 1
plt.plot(v)
print("{}, {}".format(i, Poligon.isStopped(poligon)))

In [10]:
# N droptest with random rotation and statistics
#1st set data (p, faces, dropHeight)

start_time = time.time()

N = 1000

#calculate center
#only an aproximation, it's good enough
c = np.array([0, 0, 0], dtype=float)
for a in p:
    c += np.array(a)
c /= len(p)

stat = [0] * len(faces)

for z in range(N):
    print(z)
    #set poligon
    particles = []
    for value in p:
        particles.append(Particle(value))
    poligon = Poligon(particles)
    
    #random rotation + raise to dropheight

    poligon.translate(-1 * c)

    #magic happens here
    poligon.randrotate()

    poligon.translate(c)
    poligon.translate(dropHeight)

    #droptest
    i = 0
    while (not poligon.isStopped()) and i < 10000:
        poligon.update(TIME, GRAVITY)
        i += 1

    #evaluating the droptest
    if poligon.isStopped():
        points = set()
        for i in range(poligon.n):
            if poligon.particles[i].position[2] < ZMIN:
                points.add(i)
        value = []
        for i in range(len(faces)):
            if faces[i].issubset(points):
                value.append(i)
        if len(value) == 1:
            stat[value[0]] += 1
        else:
            print('value =', value)
            print('points =', points)
    else:
        print('not stopped')
        
print(stat)
print(chisquare(stat))
print("--- %s seconds ---" % (time.time() - start_time))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

NameError: name 'chisquare' is not defined

In [ ]:
#points on ground

points = set()
for i in range(poligon.n):
    if poligon.particles[i].position[2] < 0.001:
        points.add(i)
print(points)

In [ ]:
p = [[1, 1, 1]]
datax = []
datay = []
dataz = []

for _ in range(1000):
    particles = [Particle(p[0])]
    poligon = Poligon(particles)
    
    x = random() * 2 * math.pi
    y = random() * 2 * math.pi
    z = random() * 2 * math.pi
    poligon.rotate(x, y, z)
    
    datax.append(poligon.particles[0].position[0])
    datay.append(poligon.particles[0].position[1])
    dataz.append(poligon.particles[0].position[2])
    
plt.plot(datax, datay, 'ro')
plt.axis([-2, 2, -2, 2])
plt.xlabel('x')
plt.ylabel('y')
plt.show()